<a href="https://colab.research.google.com/github/tkrdmshr/colab/blob/master/2class_all_dentsu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# ローカルファイルを直接アップロードする
from google.colab import files
uploaded = files.upload()

Saving brand_train.npy to brand_train.npy
Saving brand_val.npy to brand_val.npy
Saving x_train.npy to x_train.npy
Saving x_val.npy to x_val.npy
Saving y_train.npy to y_train.npy
Saving y_val.npy to y_val.npy


In [8]:
# ウェブサイトから直接ダウンロードする
 !wget http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2

--2019-01-16 09:15:45--  http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1373795477 (1.3G) [application/x-bzip2]
Saving to: ‘20170201.tar.bz2’

20170201.tar.bz2    100%[===================>]   1.28G  10.9MB/s    in 2m 1s   

2019-01-16 09:17:46 (10.9 MB/s) - ‘20170201.tar.bz2’ saved [1373795477/1373795477]



In [9]:
!tar -jxvf 20170201.tar.bz2

entity_vector/
entity_vector/entity_vector.model.txt
entity_vector/entity_vector.model.bin


In [12]:
# ローカルファイルを直接アップロードする
from google.colab import files
uploaded = files.upload()

Saving 2class_all.pickle to 2class_all.pickle


In [19]:
from gensim.models import KeyedVectors
import numpy as np
import pickle
from keras.layers import merge, Embedding, Flatten, Dense, Input, Bidirectional, Conv1D, MaxPooling1D, Multiply, Permute, Reshape, Concatenate
from keras.layers.recurrent import LSTM, GRU
from keras.models import Model

Using TensorFlow backend.


In [0]:
  # pickleロード用の関数
  def load_pickle(file_name):
    # loading
      with open(file_name+'.pickle', 'rb') as handle:
          return pickle.load(handle)

In [0]:
def get_embed(tokenizer):
    index2word = {v:k for k, v in tokenizer.word_index.items()}
    word_index = tokenizer.word_index
    num_words = len(word_index)
    embedding_matrix = np.zeros((num_words+1, 200))
    for word, i in word_index.items():
        if word in embeddings_model.index2word:
            try:
                embedding_matrix[i] = embeddings_model[word]
            except:
                None
    return num_words, embedding_matrix
  
  
def build_network_CNN_bi_lstm_attention(X_shape,num_words, embedding_matrix ,num_class):
    lstm_dim = 200

    '''embeddingをconcatしてCNNからのLSTMからのattention'''

    inputs = Input(shape=(X_shape,))
    embedding = Embedding(input_dim=num_words+1,output_dim=lstm_dim,weights=[embedding_matrix],trainable=False)(inputs)
    CNN_out = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(embedding)
    pool_out =MaxPooling1D(pool_size=2)(CNN_out)
    lstm_out = Bidirectional(LSTM(lstm_dim, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))(pool_out)
    attention_mul = attention_3d_block(lstm_out,int(pool_out.shape[1]))
    attention_flatten_mul = Flatten()(attention_mul)
    output = Dense(num_class, activation='sigmoid')(attention_flatten_mul)
    model = Model(inputs, output=output)
    return model
  
def attention_3d_block(inputs,time_steps):
    TIME_STEPS = time_steps
    # if True, the attention vector is shared across the input_dimensions where the attention is applied.
    SINGLE_ATTENTION_VECTOR = False

    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim,TIME_STEPS))(a)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)

    output_attention_mul = Multiply(name='attention_mul')([inputs, a_probs])
    return output_attention_mul   

  
def build_network_wrapper(shapes,words_list,embed_weights):
    model = build_network_CNN_bi_lstm_attention(shapes,words_list,embed_weights,3)
    return model


embeddings_model = KeyedVectors.load_word2vec_format("entity_vector/entity_vector.model.bin", binary= True)

In [16]:
#データのロード

x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_val = np.load('x_val.npy')
y_val = np.load('y_val.npy')
brand_train = np.load('brand_train.npy')
brand_val = np.load('brand_val.npy')

#tokenizerのロード
file_name = '2class_all'
tokenizers = load_pickle(file_name)

#モデルの構築
num_words, embedding_matrix = get_embed(tokenizers)

NameError: ignored

In [20]:
model = build_network_CNN_bi_lstm_attention(x_train.shape[1], num_words, embedding_matrix, 2)
model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("de...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 104)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 104, 200)     34107400    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 104, 32)      19232       embedding_1[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 52, 32)       0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
bidirectio

In [0]:
#学習の重み
class_weight = {0:0.50292848 ,1:85.86851852}

#学習
model.fit(x_train,y_train,
              batch_size=32,
              epochs=25,
              verbose=1, 
              shuffle=True, 
              validation_data=(x_val,y_val),
              class_weight = class_weight)

In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())